# Word2Vec experiments

## Introduction

For this project we need to learn the embeddings of a Word2Vec algorithm.
The embeddings are the weights of a single layer sequential neural network.
On this notebook we will only focus on the methods used to create these embeddings

## Skip-gram

TODO: explain method


## One-Hot encoding

TODO: explain

## Used dataset

TODO: find a dataset

## First approach : use Gensim to learn the embeddings

TODO: gensim

## Using Keras to fit a hand-crafted Word2Vec model

To better understand the underlying structure of the algorithm, we decided to implement our own neural network using Keras.

TODO: fill description

In [1]:
from text_preprocessing import NLTKTokenizer

import keras
from keras import Sequential
from keras.layers.core import Dense, Activation

import pandas as pd
import numpy as np

Using TensorFlow backend.


### Preparing the context

TODO: fill description

In [2]:
tokenizer = NLTKTokenizer('This is a test a .')


dataset = pd.Series(iter(tokenizer))


onehot = pd.get_dummies(dataset)


def test(stream):
    queue = [0, 1]
    test.count = 2
    test.vocab[next(stream)] = 0
    test.vocab[next(stream)] = 1
    yield tuple(queue)
    for token in stream:
        if token not in test.vocab:
            test.vocab[token] = test.count
            test.count += 1
        queue.append(test.vocab[token])
        if len(queue) > 3:
            queue.pop(0)
        yield tuple([queue[i] for i in [1, 0, 2]])
    queue.pop(0)
    yield((queue[1], queue[0]))


test.vocab = {}
test.count = 0


contexts = list(test(iter(tokenizer)))
print(contexts)

[(0, 1), (1, 0, 2), (2, 1, 3), (3, 2, 2), (2, 3, 4), (4, 2)]


### Keras NN model

TODO: desc

In [3]:
model = Sequential()
model.add(Dense(300, input_dim=test.count))
model.add(Activation("linear"))
model.add(Dense(test.count))
model.add(Activation("softmax"))


model.compile(optimizer=keras.optimizers.RMSprop(), loss='binary_crossentropy', metrics=['accuracy'])


### Training dataset

In [4]:

def index_to_onehot(index, n):
    res = np.zeros((1, n))
    res[0, index] = 1
    return res


def context_to_onehot(neighbors, n):
    res = np.zeros((1, n))
    for index in neighbors:
        res[0, index] = 1
    return res / len(set(neighbors))


X_train = np.array([index_to_onehot(x[0], test.count)[0] for x in contexts])
Y_train = np.array([context_to_onehot(x[1:], test.count)[0] for x in contexts])


print(Y_train)


[[0.  1.  0.  0.  0. ]
 [0.5 0.  0.5 0.  0. ]
 [0.  0.5 0.  0.5 0. ]
 [0.  0.  1.  0.  0. ]
 [0.  0.  0.  0.5 0.5]
 [0.  0.  1.  0.  0. ]]


### Performing the train

In [5]:
epoch = 15
batch_size = 32

model.summary()
model.fit(X_train, Y_train, epochs=epoch, batch_size=batch_size)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 300)               1800      
_________________________________________________________________
activation_1 (Activation)    (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1505      
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
Total params: 3,305
Trainable params: 3,305
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
6/6 [==============================] - 0s 31ms/step - loss: 0.5050 - acc: 0.7000
Epoch 2/15
6/6 [==============================] - 0s 275us/step - loss: 0.4792 - acc: 0.7000
Epoch 3/15
6/6 [==============================] - 0s 255us/step - loss: 0.4611 - acc: 0